# NYC bike crash EDA and summary stats

some code to explore the data in the NYC motor vehicle crash data set
data was downloaded from a query from here: 
https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95/data

the query parameters I used were:

column, condition, value

NUMBER OF CYCLIST INJURED, is at least, 1

NUMBER OF CYCLIST KILLED,  is at least, 1

VEHICLE TYPE CODE (1 - 5), contains, bike or bicycle

"motorbike" was sometimes returned as a match in VEHICLE TYPE


Each condition was "or'ed" with the others.  In other words, only one condition had to be true to return a row.

More data about the people in the collisions might be gotten here:
https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Person/f55k-p6yu/data

More data about the vehicles in the collisions might be gotten here:
https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Vehicles/bm4k-52h4/data

The columns are described here:  MVCollisionsDataDictionary_20190813_ERD.xlsx

In [ ]:
# the usual
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# my utilities
from crash_utils.zip_code_and_borough_from_coords import zip_code_and_borough_from_coords
from crash_utils.fix_vehicle_names import fix_vehicle_names
from crash_utils.make_crash_features import make_crash_features
from crash_utils.basic_cleaning import basic_cleaning

In [ ]:
data_path = "/Users/Mark/brainstation/capstone/nyc_bike_crash_analysis/data/"
data_file_with_path = data_path + "Motor_Vehicle_Collisions_-_Crashes.csv"
df = pd.read_csv(data_file_with_path)

In [ ]:
# fill in missing zip coded and boroughs using lat/lon
df = zip_code_and_borough_from_coords(df)

In [ ]:
# perform some basic data munging operations 
df = basic_cleaning(df)

In [ ]:
# clean up the VEHICLE TYPE CODE columns
df = fix_vehicle_names(df)

In [ ]:
# remove instances where no bike was involved
# basically, the row must contain "bike", or a cyclist was killed or injured

# the bulk of this code simply concatenates the VEHICLE TYPE columns, which is done in "make_crash_features"
# however it make_crash_features requirs the vehicle names to be fixed.  neither of these happen as early
# as when I run "basic_cleaning.py"
##  NEED TO FIX!

col_ind = df.columns.str.match("VEHICLE")
cols = df.columns[col_ind]

new_str = df["VEHICLE TYPE CODE 1"]
for col in cols[1:]:
    new_str = new_str.str.cat(df[col], sep = ",", na_rep = "")

#df.drop(columns=cols, inplace = True)
#df["VEHICLES"] = new_str


# what rows contain "bike"
# has_bike = df["VEHICLES"].str.contains("bike")
has_bike = new_str.str.contains("bike")

# maybe "bike" was recorded in vehicle types.  also check if there is a cyclist injury or death
cyclist_mask = (df["NUMBER OF CYCLIST INJURED"] > 0) | (df["NUMBER OF CYCLIST KILLED"] > 0)

# combine the masks
the_mask = has_bike | cyclist_mask

sum(~the_mask)

df = df.loc[the_mask]

In [ ]:
# add some features for easier analysis
df = make_crash_features(df, drop_featured_columns = False)

In [ ]:
df["VEHICLES"].head(20)

In [ ]:
# now encode the outcome: 0 = no injury, 1 = injury, 2 = fatality

# initiate column
df["outcome"] = np.nan

# no injuries
mask = df["NUMBER OF CYCLIST INJURED"] == 0
df.loc[mask,"outcome"] = 0

# injuries only
mask = df["NUMBER OF CYCLIST INJURED"] > 0
df.loc[mask,"outcome"] = 1

# fatalities
mask = df["NUMBER OF CYCLIST KILLED"] > 0
df.loc[mask,"outcome"] = 2


In [ ]:
#df.head()
df.columns

In [ ]:
# remove COLLISION_ID is not cross-referencing with other data files (e.g. - )
df.drop(columns="COLLISION_ID", inplace = True)

In [ ]:
print(df["DATETIME"].min())
print(df["DATETIME"].max())

In [ ]:
# write a clean data file for use with Tableau
# df.to_csv("data/cleaned_nyc_crash_table.csv", index = False)

# Summary statistics

## some basic numbers

In [ ]:
df.shape

In [ ]:
df["outcome"].value_counts().sort_index()

In [ ]:
df["NUMBER OF CYCLIST INJURED"].value_counts().sort_index()

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#df[df["NUMBER OF CYCLIST INJURED"] == 3]

In [ ]:
df["NUMBER OF CYCLIST KILLED"].value_counts().sort_values(ascending = False)

In [ ]:
# https://www.nytimes.com/2017/10/31/nyregion/police-shooting-lower-manhattan.html
df[df["NUMBER OF CYCLIST KILLED"] == 2]

## make a nice map showing accidents

https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621

In [ ]:
plt.figure(figsize=(12,12));

mask = df["NUMBER OF CYCLIST INJURED"] >0
plt.scatter(df.loc[mask,"LONGITUDE"],df.loc[mask,"LATITUDE"],4);

# mask = df["NUMBER OF CYCLIST INJURED"] == 0
# plt.scatter(df.loc[mask,"LONGITUDE"],df.loc[mask,"LATITUDE"],3,'black',alpha = 0.3);

mask = df["NUMBER OF CYCLIST KILLED"] >0
plt.scatter(df.loc[mask,"LONGITUDE"],df.loc[mask,"LATITUDE"],50,"red");

plt.show()

In [ ]:
# breakdown by borough
df_print = df.groupby(by="BOROUGH").sum()[["NUMBER OF CYCLIST KILLED","NUMBER OF CYCLIST INJURED"]].sort_values(by = "NUMBER OF CYCLIST INJURED", ascending=False)
df_print["Population"] = [2559903, 1628706, 2253858, 1418207, 476203]
df_print["injuries_per_100k"] = 1e5*df_print["NUMBER OF CYCLIST INJURED"]/df_print["Population"]
df_print["fatalaties_per_100k"] = 1e5*df_print["NUMBER OF CYCLIST KILLED"]/df_print["Population"]
df_print

In [ ]:
# streets with the most fatalities
df.groupby("ON STREET NAME").sum()["NUMBER OF CYCLIST KILLED"].sort_values(ascending=False).head(10)

In [ ]:
# streets with the most injuries
df.groupby("ON STREET NAME").sum()["NUMBER OF CYCLIST INJURED"].sort_values(ascending=False).head(10)

In [ ]:
print("Percentage of all records occuring at intersections:",round(100*df["is_intersection"].sum()/df.shape[0]),"%")
df.groupby("is_intersection").sum()[["NUMBER OF CYCLIST INJURED","NUMBER OF CYCLIST KILLED"]]

In [ ]:
#df.head()

In [ ]:
## really need to figure out how to combine the cases of "taxi bike" with "bike taxi"!!!!!!!!!!!!
df["VEHICLES"] = df["VEHICLES"].str.split().apply(np.sort).str.join(sep=" ")
df["VEHICLES"].value_counts().head(20)

In [ ]:
#df["VEHICLES"].head(20).str.split().apply(np.sort).str.join(sep=" ")

In [ ]:
df.groupby("VEHICLES").sum()[["NUMBER OF CYCLIST INJURED","NUMBER OF CYCLIST KILLED"]].sort_values(by="NUMBER OF CYCLIST KILLED", ascending = False).head(30)

In [ ]:
df.groupby("factors").sum()[["NUMBER OF CYCLIST INJURED","NUMBER OF CYCLIST KILLED"]].sort_values(by="NUMBER OF CYCLIST KILLED", ascending = False).head(30)

## time series!

In [ ]:
df.head()

In [ ]:
# hey let's plot a time series!
df.resample('M',on="DATETIME",loffset="-15D").sum()["NUMBER OF CYCLIST INJURED"].head(10)

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(df.resample('M',on="DATETIME",loffset="-15D").sum()["NUMBER OF CYCLIST INJURED"],marker='o')
plt.ylabel("Cyclists injured per month", size = 18);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);
plt.title("Injuries, 2012 - 2020", size = 20);
plt.show()

plt.figure(figsize=(15,6))
plt.plot(df.resample('M',on="DATETIME",loffset="-15D").sum()["NUMBER OF CYCLIST KILLED"],marker='o')
plt.ylabel("Cyclists killed per month", size = 18);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);
plt.title("Fatalities, 2012 - 2020", size = 20);
plt.show()

In [ ]:
plt.figure()
df.groupby("MONTH").sum()["NUMBER OF CYCLIST INJURED"].plot.bar(figsize=(10,6));
plt.title("Monthly sum of injuries", size = 16)
plt.ylabel("Number of injuries", size = 16);
plt.xlabel("Month", size=16)
plt.xticks(size=16)
plt.yticks(size=16)
plt.show()

plt.figure()
df.groupby("MONTH").sum()["NUMBER OF CYCLIST KILLED"].plot.bar(figsize=(10,6));
plt.title("Monthly sum of deaths", size = 16)
plt.ylabel("Number of fatalities", size = 16);
plt.xlabel("Month", size=16)
plt.xticks(size=16)
plt.yticks(size=16)
plt.show()

In [ ]:
# Monday is day 0
plt.figure()
df.groupby("DAY_OF_WEEK").sum()["NUMBER OF CYCLIST INJURED"].plot.bar(figsize=(10,6));
plt.title("Number of injuries by day", size = 16)
plt.xlabel("Day of week (0 = Monday)", size = 16);
plt.ylabel("Number of injuries", size = 16);
plt.xticks(size=16);
plt.yticks(size=16);
plt.show()

plt.figure()
df.groupby("DAY_OF_WEEK").sum()["NUMBER OF CYCLIST KILLED"].plot.bar(figsize=(10,6));
plt.title("Number of fatalities by day", size = 16)
plt.xlabel("Day of week (0 = Monday)", size = 16);
plt.ylabel("Number of fatalities", size = 16);
plt.xticks(size=16);
plt.yticks(size=16);
plt.show()

In [ ]:
# Hour of day
plt.figure()
df.groupby("HOUR").sum()["NUMBER OF CYCLIST INJURED"].plot.bar(figsize=(10,6));
plt.title("Number of injuries by hour", size = 16)
plt.xlabel("Hour (0 = midnight)", size = 16);
plt.ylabel("Number of injuries", size = 16);
plt.xticks(size=16);
plt.yticks(size=16);
plt.show()

plt.figure()
df.groupby("HOUR").sum()["NUMBER OF CYCLIST KILLED"].plot.bar(figsize=(10,6));
plt.title("Number of fatalities by day", size = 16)
plt.xlabel("Hour (0 = midnight)", size = 16);
plt.ylabel("Number of fatalities", size = 16);
plt.xticks(size=16);
plt.yticks(size=16);
plt.show()

In [ ]:
df.columns